In [2]:
!unzip '/content/drive/My Drive/NNProjectk.zip'

Streaming output truncated to the last 5000 lines.
  inflating: NNProject/train/n04067472/images/n04067472_73.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_72.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_71.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_70.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_7.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_69.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_68.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_67.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_66.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_65.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_64.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_63.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_62.JPEG  
  inflating: NNProject/train/n04067472/images/n04067472_61.JPEG  
  inflating: NNProject/tra

**This model took a lot of traning time for one epoch. And Accuracy is same as of Inception V1 model coded from scratch in another file.**

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, activations
import os
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import itertools
#import shutil
%matplotlib inline

In [4]:


tr_datagen =  ImageDataGenerator(rescale=1.0/255, 
                                 horizontal_flip=True,
                                 vertical_flip=True)
train_batch_size = 32
val_batch_size= 32
IMAGE_SIZE= 224
train_path ="NNProject/train/"

train_gen = tr_datagen.flow_from_directory(train_path,
                                target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                batch_size=train_batch_size,
                                class_mode='categorical')


val_datagen = ImageDataGenerator(rescale=1./255)
val_data = pd.read_csv('NNProject/val/val_annotations.txt', sep='\t',header=None,names=['File','Class','X','Y','H','W'])
val_data.drop(['X','Y','H','W'], axis=1, inplace=True)
val_data.head()


val = val_datagen.flow_from_dataframe(val_data,"NNProject/val/images", 
                                      x_col='File', y_col ='Class' ,
                                      class_mode="categorical",
                                      batch_size=40,
                                      target_size=(224, 224))



Found 90000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [5]:
def conv2d(x,numfilt,filtsz,strides=1,pad='same',act=True,name=None):
  x = Conv2D(numfilt,filtsz,strides,padding=pad,data_format='channels_last',use_bias=False,name=name+'conv2d')(x)
  x = BatchNormalization(axis=3,scale=False,name=name+'conv2d'+'bn')(x)
  if act:
    x = Activation('relu',name=name+'conv2d'+'act')(x)
  return x

In [6]:
##IRA
def incresA(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,32,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,32,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,32,3,1,pad,True,name=name+'b1_2')
    branch2 = conv2d(x,32,1,1,pad,True,name=name+'b2_1')
    branch2 = conv2d(branch2,48,3,1,pad,True,name=name+'b2_2')
    branch2 = conv2d(branch2,64,3,1,pad,True,name=name+'b2_3')
    branches = [branch0,branch1,branch2]
    mixed = Concatenate(axis=3, name=name + '_concat')(branches)
    filt_exp_1x1 = conv2d(mixed,384,1,1,pad,False,name=name+'filt_exp_1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=backend.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_scaling')([x, filt_exp_1x1])
    return final_lay

In [7]:
##IRB
def incresB(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,192,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,128,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,160,[1,7],1,pad,True,name=name+'b1_2')
    branch1 = conv2d(branch1,192,[7,1],1,pad,True,name=name+'b1_3')
    branches = [branch0,branch1]
    mixed = Concatenate(axis=3, name=name + '_mixed')(branches)
    filt_exp_1x1 = conv2d(mixed,1152,1,1,pad,False,name=name+'filt_exp_1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=backend.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_scaling')([x, filt_exp_1x1])
    return final_lay

In [8]:
##IRC
def incresC(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,192,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,192,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,224,[1,3],1,pad,True,name=name+'b1_2')
    branch1 = conv2d(branch1,256,[3,1],1,pad,True,name=name+'b1_3')
    branches = [branch0,branch1]
    mixed = Concatenate(axis=3, name=name + '_mixed')(branches)
    filt_exp_1x1 = conv2d(mixed,2048,1,1,pad,False,name=name+'fin1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=backend.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_saling')([x, filt_exp_1x1])
    return final_lay

In [9]:
##STEM
img_input = Input(shape=(224,224,3))

x = conv2d(img_input,32,3,2,'valid',True,name='conv1')
x = conv2d(x,32,3,1,'valid',True,name='conv2')
x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x_12 = conv2d(x,64,3,1,'valid',True,name='stem_br_12')

x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

x_31 = conv2d(x,192,3,2,'valid',True,name='stem_br_31')
x_32 = MaxPooling2D(3,strides=2,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])

In [10]:
#Inception-ResNet-A modules
x = incresA(x,0.15,name='incresA_1')
x = incresA(x,0.15,name='incresA_2')
x = incresA(x,0.15,name='incresA_3')
x = incresA(x,0.15,name='incresA_4')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

#Inception-ResNet-B modules
x = incresB(x,0.1,name='incresB_1')
x = incresB(x,0.1,name='incresB_2')
x = incresB(x,0.1,name='incresB_3')
x = incresB(x,0.1,name='incresB_4')
x = incresB(x,0.1,name='incresB_5')
x = incresB(x,0.1,name='incresB_6')
x = incresB(x,0.1,name='incresB_7')

#17 × 17 to 8 × 8 reduction module.
x_red_21 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

x_red_22 = conv2d(x,256,1,1,'same',True,name='x_red2_c11')
x_red_22 = conv2d(x_red_22,384,3,2,'valid',True,name='x_red2_c12')

x_red_23 = conv2d(x,256,1,1,'same',True,name='x_red2_c21')
x_red_23 = conv2d(x_red_23,256,3,2,'valid',True,name='x_red2_c22')

x_red_24 = conv2d(x,256,1,1,'same',True,name='x_red2_c31')
x_red_24 = conv2d(x_red_24,256,3,1,'same',True,name='x_red2_c32')
x_red_24 = conv2d(x_red_24,256,3,2,'valid',True,name='x_red2_c33')

x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])

#Inception-ResNet-C modules
x = incresC(x,0.2,name='incresC_1')
x = incresC(x,0.2,name='incresC_2')
x = incresC(x,0.2,name='incresC_3')

#TOP
x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dropout(0.6)(x)
x = Dense(200, activation='softmax')(x)

In [11]:
model = Model(img_input,x,name="inception_resnet_v2")
model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 111, 111, 32) 96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 111, 111, 32) 0           conv1conv2dbn[0][0]              
________________________________________________________________________________

In [12]:
model.compile(Adam(lr=0.0001), 
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [13]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", 
                        verbose=1, save_best_only=True, mode="max")

In [14]:
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=4, restore_best_weights=True)
callbacks_list = [checkpoint, early]


In [ ]:
history = model.fit(train_gen,
                              steps_per_epoch=2812,          
                              validation_data=val,
                              validation_steps=312, 
                              epochs=2, verbose=1,
                              callbacks=callbacks_list)

**Prediction on Test data can be coded further. This was just to get the idea of architecture and accuracy of the model from scratch.**